In [2]:
from sympy import *

In [3]:
def Rx(th):
    return Matrix([
        [1, 0, 0],
        [0, cos(th), -sin(th)],
        [0, sin(th), cos(th)]
    ])
def Ry(th):
    return Matrix([
        [cos(th), 0, sin(th)],
        [0, 1, 0],
        [-sin(th), 0, cos(th)]
    ])
def Rz(th):
    return Matrix([
        [cos(th), -sin(th), 0],
        [sin(th), cos(th), 0],
        [0, 0, 1]
    ])

![agile eye](agile_eye.jpg)

In [4]:
#joint angles in the agile eye as shown in the sketch
th1, th2, th3, th4, th5 = symbols('θ1 θ2 θ3 θ4 θ5')

#the longer agile arm chain rotates about z, then x, then y
R_2arm = trigsimp(Rz(th1)@Rx(th2)@Ry(th3))
R_2arm

Matrix([
[-sin(θ1)*sin(θ2)*sin(θ3) + cos(θ1)*cos(θ3), -sin(θ1)*cos(θ2), sin(θ1)*sin(θ2)*cos(θ3) + sin(θ3)*cos(θ1)],
[ sin(θ1)*cos(θ3) + sin(θ2)*sin(θ3)*cos(θ1),  cos(θ1)*cos(θ2), sin(θ1)*sin(θ3) - sin(θ2)*cos(θ1)*cos(θ3)],
[                          -sin(θ3)*cos(θ2),          sin(θ2),                           cos(θ2)*cos(θ3)]])

In [5]:
#shorter chain rotates about y then z
R_1arm = trigsimp(Ry(th4)@Rz(th5))
R_1arm

Matrix([
[ cos(θ4)*cos(θ5), -sin(θ5)*cos(θ4), sin(θ4)],
[         sin(θ5),          cos(θ5),       0],
[-sin(θ4)*cos(θ5),  sin(θ4)*sin(θ5), cos(θ4)]])

In [28]:
thz, thy = symbols('θ_z θ_y') #IMU yaw and pitch
R_cam = Rz(thz)@Ry(thy)
f1 = R_cam - R_1arm #solve for angles on shorter chain by setting equal to camera orientation
solcam45 = solve(f1, [th4,th5])
solcam45

[(θ_y, θ_z)]

In [12]:
R_1arm.subs({th4:0.2,th5:0.3})

Matrix([
[ 0.936293363584199, -0.289629477625516, 0.198669330795061],
[  0.29552020666134,  0.955336489125606,                 0],
[-0.189796060978687, 0.0587108016938265, 0.980066577841242]])

In [34]:
m_y = atan2(-R_cam[2,0], R_cam[0,0]).subs({thz:0.2,thy:0.3})
az = asin(R_cam[1,0]).subs({thz:0.2,thy:0.3})
print(m_y, az)

0.305731843866098 0.190954427822008


In [35]:
R_1arm.subs({th4:m_y,th5:az})

Matrix([
[0.936293363584199, -0.180994634849206, 0.300991161770556],
[0.189796060978687,  0.981823535691101,                 0],
[-0.29552020666134, 0.0571269368934505, 0.953626929431007]])

In [8]:
f2 = R_cam - R_2arm #solve for angles on longer chain by setting equal to camera orientation
solcam123 = solve(f2, [th1,th2,th3])
solcam123

[(pi - asin(sin(θ_z)), 0, pi - asin(sin(θ_y))),
 (pi - asin(sin(θ_z)), 0, asin(sin(θ_y))),
 (asin(sin(θ_z)) + pi, pi, asin(sin(θ_y)) + pi),
 (asin(sin(θ_z)) + pi, pi, -asin(sin(θ_y))),
 (-asin(sin(θ_z)), pi, asin(sin(θ_y)) + pi),
 (-asin(sin(θ_z)), pi, -asin(sin(θ_y))),
 (asin(sin(θ_z)), 0, pi - asin(sin(θ_y))),
 (asin(sin(θ_z)), 0, asin(sin(θ_y)))]

In [9]:
test_dict = {thz: -0.4, thy: 0.3}
for i in range(len(solcam123)):
    th1_n = solcam123[i][0].subs(test_dict).n()
    th2_n = solcam123[i][1].subs(test_dict).n()
    th3_n = solcam123[i][2].subs(test_dict).n()
    print(th1_n, th2_n, th3_n)
#last solution seems most reasonable

3.54159265358979 0 2.84159265358979
3.54159265358979 0 0.300000000000000
2.74159265358979 3.14159265358979 3.44159265358979
2.74159265358979 3.14159265358979 -0.300000000000000
0.400000000000000 3.14159265358979 3.44159265358979
0.400000000000000 3.14159265358979 -0.300000000000000
-0.400000000000000 0 2.84159265358979
-0.400000000000000 0 0.300000000000000


In [41]:
import numpy as np
#sweep angles in front hemisphere
for thz_n in np.linspace(-np.pi/2, np.pi/2, 10):
    for thy_n in np.linspace(-np.pi/2, np.pi/2, 10):
        test_dict = {thz: thz_n, thy: thy_n}
        th1_n = solcam123[7][0].subs(test_dict).n()
        th4_n = solcam45[0][0].subs(test_dict).n()
        # print(thz_n, th1_n, thy_n, th4_n)
        print(thz_n-th1_n, thy_n-th4_n)
        print(f1.subs({thz: thz_n, thy: thy_n, th1:solcam123[7][0].subs(test_dict).n(),th2:solcam123[7][1].subs(test_dict).n(),th3:solcam123[7][2].subs(test_dict).n(), th4:solcam45[0][0].subs(test_dict).n(), th5:solcam45[0][1].subs(test_dict).n()}))

0 0
Matrix([[0, 1.00000000000000, 1.00000000000000], [1.00000000000000, 0, 1.00000000000000], [1.00000000000000, -1.00000000000000, 0]])
0 0
Matrix([[0, 0.657979856674331, 0.939692620785908], [0.657979856674331, 0, 0.939692620785908], [0.939692620785908, -0.939692620785908, 0]])
0 0
Matrix([[0, 0.357212390313461, 0.766044443118978], [0.357212390313461, 0, 0.766044443118978], [0.766044443118978, -0.766044443118978, 0]])
0 0
Matrix([[0, 0.133974596215561, 0.500000000000000], [0.133974596215561, 0, 0.500000000000000], [0.500000000000000, -0.500000000000000, 0]])
0 0
Matrix([[0, 0.0151922469877920, 0.173648177666930], [0.0151922469877920, 0, 0.173648177666930], [0.173648177666930, -0.173648177666930, 0]])
0 0
Matrix([[0, 0.0151922469877920, -0.173648177666930], [0.0151922469877920, 0, -0.173648177666930], [-0.173648177666930, 0.173648177666930, 0]])
0 0
Matrix([[0, 0.133974596215561, -0.500000000000000], [0.133974596215561, 0, -0.500000000000000], [-0.500000000000000, 0.500000000000000, 0]